In [2]:
pip install mysql-connector-python

Note: you may need to restart the kernel to use updated packages.


In [3]:
import mysql.connector

def connect_to_database():
    try:
        connection = mysql.connector.connect(
            host='localhost',         # Replace with your database host
            user='your_username',     # Replace with your MySQL username
            password='your_password', # Replace with your MySQL password
            database='sakila'         # Name of the database
        )
        if connection.is_connected():
            print("Connected to the Sakila database")
            return connection

    except mysql.connector.Error as err:
        print(f"Error: {err}")
        return None

conn = connect_to_database()

if conn:
    cursor = conn.cursor()
    cursor.execute("SELECT NOW();")
    result = cursor.fetchone()
    print("Database server time:", result[0])
    cursor.close()
    conn.close()

Error: 1045 (28000): Access denied for user 'your_username'@'localhost' (using password: YES)


In [4]:
import pandas as pd
from sqlalchemy import create_engine
from datetime import datetime

def rentals_month(engine, month, year):
    # Prepare the connection and query
    connection = engine.connect()
    query = f"""
    SELECT * FROM rental 
    WHERE MONTH(rental_date) = {month} AND YEAR(rental_date) = {year};
    """
    df = pd.read_sql(query, connection)
    connection.close()
    return df

In [5]:
def rental_count_month(rentals_df, month, year):
    rental_counts = rentals_df.groupby('customer_id').size().reset_index(name=f'rentals_{month:02}_{year}')
    return rental_counts

In [6]:
def compare_rentals(df1, df2):
    # Merge two DataFrames on the customer_id ignoring missing values for merging
    combined = pd.merge(df1, df2, on='customer_id', how='outer').fillna(0)
    

    combined['difference'] = combined.iloc[:, 1] - combined.iloc[:, 2]
    return combined